In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using Plots
using Test
include("car.jl");
using ForwardDiff

  Activating environment at `c:\Users\ADMIN\Documents\GitHub\ocrl_project\Project.toml`
  Activating environment at `c:\Users\ADMIN\Documents\GitHub\ocrl_project\Project.toml`


In [2]:
# Define the model
model = BicycleModel()

# get the number of states and controls
n = state_dim(model)
m = control_dim(model)

# Evaluate the continuous and discrete dynamics
x0 = SA[0.,0.,0.,0.]
u0 = SA[0.,0.]
t0 = 0.0
dt = 0.025
dynamics(model, x0, u0)
discrete_dynamics(RK4, model, x0, u0, t0, dt)  # use rk4 for integration

# Evaluate the continuous and discrete Jacobians
z0 = KnotPoint(x0,u0,dt,t0)   # create a `KnotPoint` type that stores everything together
∇f = RobotDynamics.DynamicsJacobian(model)
jacobian!(∇f, model, z0)
discrete_jacobian!(RK4, ∇f, model, z0)

# Extract pieces of the Jacobian
A = ∇f.A
B = ∇f.B;

In [3]:
nx = n
nu = m

Q = Diagonal([1,1,1e-2,1e-2])
R = Diagonal([1e-1,1e-1])
Qf = Diagonal([1,1,1,1.])*10

N = 101 #401
Nmpc = 10 #60
Nhz = 1
delta = 12 #80
Nc = N + 2*delta

FD = ForwardDiff;

In [213]:
function get__vanilla_trajectory(s)
    
    arr = split(s,"")
    if 60 < length(arr) < 120
        c = 2
    elseif length(arr) > 120
        c = 1
    else 
        c = 3
    end 

    N = length(arr) * c
   

    x1ref = zeros(N)
    x2ref = zeros(N)
    θref = zeros(N)
    v1ref = zeros(N)
    v2ref = zeros(N)
    ωref = zeros(N)
    rad = 20

    u1ref = zeros(N-1)*NaN
    u2ref = zeros(N-1)*NaN

    #v1ref[1] = 0.0
    #v1ref[Nc] = v1ref[Nc-1]
    @show(size(θref))
    counter = 0
    for i in 1:length(arr)
        
        if arr[i] == "s"
            x1ref[i] += 0
            x2ref[i] += 0
            θref[i] += 0
        elseif arr[i] == "w"
            x2ref[i] +=1
            
        elseif arr[i] == "a"
            x1ref[i] += 1
        elseif arr[i] =="d"
            x1ref[i] -=1
        end
        
    end
    xref = [x1ref'; x2ref'; θref'; v1ref']
    uref = [u1ref'; u2ref']
    return [x for x in eachcol(xref)], [u for u in eachcol(uref)], length([x for x in eachcol(xref)])
end

get__vanilla_trajectory (generic function with 1 method)

In [214]:
using Interpolations
using Plots

In [215]:
function Linear_Fit_between(knot1,knot2,steps)
    if steps == 0
        return 0
    end

    if steps == 1
        return [knot1]
    end

    delta = (knot2 - knot1)/(steps)
    #@show(delta)
    array = zeros(steps)
    #@show(size(array))
    array[1] = knot1
    for i in 1:steps-1
        array[i+1] = array[i] + delta
    end
    return array
end

Linear_Fit_between (generic function with 1 method)

In [229]:
function get_complex_traj(input)
    arr = split(input,"")
    #println(arr)
    if 60 < length(arr) < 120
        c = 2
    elseif length(arr) > 120
        c = 1
    else 
        c = 3
    end

    c = c*4
    N = ((length(arr)÷ 4))*c 
    @show N 
    if N == 0
        return zeros(12,4)
    end
    
    x1ref = zeros(N)*NaN # x-Coord
    x2ref = zeros(N)*NaN # y-Coord
    θref = zeros(N)*NaN # Angular (tan-1(y/x))
    v1ref = zeros(N)*NaN
    v2ref = zeros(N)*NaN
    ωref = zeros(N)*NaN
    rad = 20

    u1ref = zeros(N-1)
    u2ref = zeros(N-1)

    block = zeros((length(arr) ÷ 4),4)*NaN
    #@show size(block)
    
    counter  = 1
    row_count = 1

    xc = 0.0
    yc = 0.0
    tc = 0.0
    vc = 0.0

    Nb,Cb = size(block)
    #@show(Nb)

    for i in 1:length(arr)
        #@show(xc)
        if arr[i] == "s"
            continue
        elseif arr[i] == "w"
            xc = xc + 0.0
            yc = yc + 1.0
        elseif arr[i] == "a"
            xc = xc - 1.0
            yc = yc + 0.0
        elseif arr[i] == "d"
            xc = xc + 1.0
            yc = yc + 0.0
        elseif arr[i] == "q"
            xc -=1.0/sqrt(2.0)
            yc +=1.0/sqrt(2.0)
        elseif arr[i] == "e"
            xc = xc + 1.0
            yc = yc + 1.0
        end
        #@show(yc)

        
        if counter == 4
            counter = 0
            
            block[row_count,1] = xc
            block[row_count,2] = yc
            block[row_count,3] = tc
            block[row_count,4] = vc

            if row_count > Nb 
                break
            end
            
            row_count +=1
            #@show(row_count)
        end
        
        counter +=1
    end
    #@show(block)
    
    start_vec = zeros(1,4)
    x1ref[1:c] .= Linear_Fit_between(0,block[1,1],c)
    x2ref[1:c] .= Linear_Fit_between(0,block[1,2],c)

    for i = 2:Nb
        start = (i-1)*c + 1
        finish = i*c
        #@show(c)
        #@show(finish)
        #@show block[i,1]
        #@show block[i+1,1]
        arrx = Linear_Fit_between(block[i-1,1],block[i,1],c)
        arry = Linear_Fit_between(block[i-1,2],block[i,2],c)
        #@show size(arrx)

        x1ref[start:finish] .= arrx
        x2ref[start:finish] .= arry
    end
    x1ref[end] = block[Nb,1]
    x2ref[end] = block[Nb,2]
    #@show x1ref

    #@show size(x1ref)
    θref[1] = 0
    u1ref[1] = 0
    v1ref[1] = 0
    u2ref[1] = 0
    for i = 2:N
        if (x1ref[i]-x1ref[i-1]) == 0
            θref[i] = (pi/2)
        else
            θref[i] = atan(x2ref[i]-x2ref[i-1])/((x1ref[i]-x1ref[i-1]))
        end
        
        u1ref[i-1] = sqrt((x2ref[i]-x2ref[i-1])^2 + (x1ref[i]-x1ref[i-1])^2)
        v1ref[i] = 0
        u2ref[i-1] = 0
    end

    xref = [x1ref'; x2ref'; θref'; v1ref']
    uref = [u1ref'; u2ref']
    #@show size([x for x in eachcol(xref)])
    return [x for x in eachcol(xref)], [u for u in eachcol(uref)]
end

get_complex_traj (generic function with 1 method)

In [230]:
in = readline()
get_complex_traj(in)

N = 72


(SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}[[0.0, 0.0, 0.0, 0.0], [0.16666666666666666, 0.25, 1.469871978761185, 0.0], [0.3333333333333333, 0.5, 1.469871978761185, 0.0], [0.5, 0.75, 1.4698719787611847, 0.0], [0.6666666666666666, 1.0, 1.4698719787611851, 0.0], [0.8333333333333333, 1.25, 1.4698719787611851, 0.0], [0.9999999999999999, 1.5, 1.4698719787611851, 0.0], [1.1666666666666665, 1.75, 1.4698719787611851, 0.0], [1.3333333333333333, 2.0, 1.4698719787611843, 0.0], [1.5, 2.25, 1.4698719787611843, 0.0]  …  [12.333333333333332, 13.5, 1.46987197876119, 0.0], [12.499999999999998, 13.75, 1.46987197876119, 0.0], [12.666666666666664, 14.0, 1.46987197876119, 0.0], [12.83333333333333, 14.25, 1.46987197876119, 0.0], [12.999999999999996, 14.5, 1.46987197876119, 0.0], [13.166666666666663, 14.75, 1.46987197876119, 0.0], [13.333333333333329, 15.0, 1.46987197876119, 0.0], [13.499999999999995, 15.25, 1.46987197876119, 0.0], [13.66666666666666, 15.5, 1.4698

In [231]:
in = readline()

Xref, Uref = get_complex_traj(in)
# convert to static arrays and plot
Xref = [SVector{4}(Xref[i]) for i = 1:length(Xref)]
tref = SA[0:dt:(Nc-1-2*delta)*dt]
trefu = SA[0:dt:(Nc-2-2*delta)*dt]

Uref = [SVector{2}(Uref[i]) for i = 1:length(Uref)];

N = 108


In [232]:
Q = Diagonal([10,10,10,1.])
R = Diagonal([1.,1.])
Qf = Diagonal([1,1,1,1.])*1;

In [233]:
function stage_cost(x,u,xref,uref, Q, R, Qf)
    # LQR cost at each knot point (depends on both x and u)
    J = 0.5 * (x - xref)' * Q * (x - xref) + 0.5 * (u - uref)' * R * (u - uref)
    
    return J
end

function term_cost(x,xref, Q, R, Qf)
    # LQR terminal cost (depends on just x)
    J = 0.5 * (x - xref)' * Qf * (x - xref)
    
    return J
end

function trajectory_cost(X,U,Xref,Uref, Q, R, Qf, Nmpc)
    # calculate the cost of a given trajectory 
    J = 0.0
    for i = 1:Nmpc-1
        J += stage_cost(X[i],U[i],Xref[i],Uref[i], Q, R, Qf)
    end
    J += term_cost(X[end], Xref[end], Q, R, Qf)
    
    return J
end
        
function stage_cost_expansion(x,u,xref,uref, Q, R, Qf, Nmpc)
    # if the stage cost function is J, return the following derivatives:
    # ∇²ₓJ,  ∇ₓJ, ∇²ᵤJ, ∇ᵤJ
    Jxx = Q
    Jx = Q * (x - xref)
    Juu = R
    Ju = R * (u - uref)
    
    return Jxx, Jx, Juu, Ju
end

function term_cost_expansion(x,xref, Q, R, Qf, Nmpc)
    # if the terminal cost function is J, return the following derivatives:
    # ∇²ₓJ,  ∇ₓJ
    Jxx = Qf
    Jx = Qf * (x - xref)
    
    return Jxx, Jx
end

term_cost_expansion (generic function with 1 method)

In [234]:
function stage_cost_convoy(x, xref, Q)
    # LQR cost at each knot point (depends on both x and u)
    J = 0.5 * (x - xref)' * Q * (x - xref)
    
    return J
end

function trajectory_cost_convoy(X, Xref, Q, Nmpc)
    # calculate the cost of a given trajectory 
    J = 0.0
    for i = 1:Nmpc
        J += stage_cost_convoy(X[i], Xref[i], Q)
    end
    
    return J
end
        
function stage_cost_expansion_convoy(x, xref, Q, Nmpc)
    # if the stage cost function is J, return the following derivatives:
    # ∇²ₓJ,  ∇ₓJ, ∇²ᵤJ, ∇ᵤJ
    Jxx = Q
    Jx = Q * (x - xref)
    
    return Jxx, Jx
end

function total_cost(X,U,Xref,Uref,Xfoll,Xlead, K,d,ΔJ, Q, R, Qf, Qfoll, Qlead, Nmpc)
    J_traj = trajectory_cost(X,U,Xref,Uref, Q, R, Qf, Nmpc)
    J_foll = trajectory_cost_convoy(X, Xfoll, Qfoll, Nmpc)
    J_lead = trajectory_cost_convoy(X, Xlead, Qlead, Nmpc)
    return J_traj + J_foll + J_lead
end

total_cost (generic function with 1 method)